In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import time
import utils
from utils import *
import os
import importlib
import argparse
import trader   # 파일 수정 반영
import matplotlib.pyplot as plt

In [2]:
binance = get_binance()

In [3]:
tf = "1m"
limit = 1500
past_df_dic = {}
new_symlist = []
for sym in SYMLIST[0:50]:
    vol = await binance.fetch_tickers(symbols=[sym])
    if (not len(list(vol.values())) > 0) or list(vol.values())[0]['quoteVolume'] < 20*(10**6):
        continue
    df = await past_data(binance, sym, tf, limit)
    df["mean"] = np.mean([df["open"], df["high"], df["low"], df["close"]], axis=0)
    past_df_dic.update({sym: df})
    new_symlist.append(sym)

In [64]:
def bull_or_bear(df,ref=0.01):
    m = (df['high']+df['low'])/2  # df['close']로 해도 되는데 그냥 이렇게 함
    rising = []
    for i in range(1, len(m)-1):
        rising.append((m[i+1] - m[i])/m[i]*100)
    rising_coef = np.mean(rising)
    
    if rising_coef > ref:
        return "BULL", rising_coef
    elif rising_coef < -ref:
        return "BEAR", rising_coef
    else:
        return "ZZ", rising_coef
    
def inspect_market(df, block):
    st1, score1 = bull_or_bear(df[int(block/4):])
    st2, score2 = bull_or_bear(df)
    score = score1
    
    if st2 == 'BEAR' and st1 == "BULL":
        return LONG, score
    elif st2 == 'BULL' and st1 == "BEAR":         # 4시간동안 상승
        return SHORT, score
    return None, 0

In [65]:
def select_sym(t, block):
    max_score, min_score = 0,0
    max_sym, min_sym = 0,0
    for sym in new_symlist:
        df = past_df_dic[sym][t-block:t]
        position, score = inspect_market(df, block)
        
        if score > max_score:
            max_score, max_sym = score, sym
        elif score < min_score:
            min_score, min_sym = score, sym

    if (abs(max_score) > 0 or abs(min_score) > 0 ) and position:
        print(f"== MAX: {max_sym} {max_score} | MIN: {min_sym} {min_score} ==")
        if abs(max_score) > abs(min_score):
            return max_sym, position
        else:
            return min_sym, position 
    return None, None

In [66]:

def get_pnl(open_price, price):
    return (price - open_price)/open_price

def run(t, sl, tp, lev, block):
    sym, position = select_sym(t, block)
    if not sym and not position:
        return 0,0,0,0,0,0
    sym_df = past_df_dic[sym]['mean'][t:]

    open_price = sym_df[0]
    
    for i, mean_price in enumerate(sym_df[1:]):
        coef = 1 if position == "Long" else -1
        pnl = get_pnl(open_price, mean_price) * lev * coef * 100 - 0.7
        if pnl > tp or pnl < sl:
            # print(open_price, mean_price, pnl, position)
            return i+1, open_price, mean_price, pnl, sym, position
        
    return 0,0,0,0,0,0
        
def show_default_graph(df, t, time_i, open_price, close_price, position, title=""):
    plt.style.use('fivethirtyeight')
    fig, ax = plt.subplots(figsize = (12,6))

    x = len(df)
    
    ax.plot(range(x), df, color = (1, 0.7, 0), linewidth = 2, label='mean price')
        
    color = 'b' if position == 'Short' else 'r'
    ax.add_patch(plt.Arrow(
        t, open_price, time_i, close_price-open_price,
        width = (close_price-open_price)*2, color=color, zorder=2, alpha=0.5
    ))
        
    ax.set_xlabel('Date')
    ax.set_ylabel('Price ($)')

    ax.set_title(title)
    os.makedirs("backtest_result", exist_ok=True)
    plt.savefig(f"backtest_result/{t}.jpg")
    return ax

def simul(pnls):
    orig = 100
    profit = 0
    for pnl in pnls:
        if pnl != 0:
            # profit += orig*(pnl*0.01)
            orig *= (1+pnl*0.01)
    return orig+profit

In [67]:
sl, tp = -30, 4
lev = 2
pnls = []
block = 50

for t in range(block, 1500, 10):
    time_i, open_price, close_price, pnl, sym, position = run(t, sl, tp, lev, block)
    pnls.append(pnl)
    if pnl != 0:
        show_default_graph(past_df_dic[sym]['mean'], t, time_i, open_price, close_price, position, title=f"{sym}  PNL: {pnl}%")
        
print(np.mean(pnls))

== MAX: DYM/USDT 0.02670918432074985 | MIN: 0 0 ==
0.0


In [68]:
pnls = np.array(pnls)
print(f"Total PNL: {np.sum(pnls)}%,    # of good pred: {np.sum(np.where(pnls>0, 1, 0))},    bad pred: {np.sum(np.where(pnls<0, 1, 0))}")
simul(pnls)

Total PNL: 0%,    # of good pred: 0,    bad pred: 0


100